In [1]:
import json
import numpy as np
from utils.transform_utils import quat2mat, convert_quat
import cv2
from scipy.spatial.transform import Rotation

import sys
# sys.path.append("/home/vdrame/catkin_ws/src/py_panda/PyPanda")
from PyPanda import Robot
import rospy

from utils.camera_utils import RealCamera, RealCameraROS

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-07-04 14:22:31,229 - utils - NumExpr defaulting to 8 threads.
INFO - 2023-07-04 14:22:32,043 - topics - topicmanager initialized


# viewpoint registering

In [2]:
rospy.init_node("pythonnode_register_views",anonymous=True)
rbt = Robot("panda", use_gripper=True)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

# views_pos = []
# views_orn_wxyz = []
views = {}
view_idx = 0
while True:
    bgr, depth_image, depth_scale = camera.get_rgb_depth()
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    cv2.imshow("rgb", rgb)
    key = cv2.waitKey(1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    elif key == ord('s'):
        view_pos = rbt.model.ee_pos_rel()
        view_orn_wxyz = rbt.model.ee_orn_rel()
        views["view"+str(view_idx)] = {"pos": view_pos.tolist(), "orn_wxyz": view_orn_wxyz.tolist()}
        print("view{} saved".format(view_idx))
        view_idx += 1


print(views)

#write pos in json file
with open('config/views_pos.json', 'w') as outfile:
    json.dump(views, outfile, indent=1)

rospy already initialized
Camera topic found
view0 saved
view1 saved
view2 saved
view3 saved
{'view0': {'pos': [0.3337424752486864, -0.13824362930022224, 0.57657893633769], 'orn_wxyz': [-0.013550037090920081, 0.8976251173601688, -0.4404160815328442, -0.010918805165078739]}, 'view1': {'pos': [0.019865728789256305, -0.4189516988934032, 0.43480189553046517], 'orn_wxyz': [-0.31186908072882913, 0.8486066443313943, -0.40244327827278, 0.14367966959533035]}, 'view2': {'pos': [0.5293927876015859, 0.029630033453038877, 0.46389673092862843], 'orn_wxyz': [0.21118281924603413, 0.8596909565204475, -0.4440077730254348, -0.13852932406262897]}, 'view3': {'pos': [0.16733281486867313, 0.13695803210945723, 0.534313727211819], 'orn_wxyz': [0.10419839319946969, 0.8801648337269176, -0.41653112619747945, 0.20237188844742177]}}


In [3]:
#load pos and orn from json file
with open('config/views_pos.json') as json_file:
    views_pos_1 = json.load(json_file)

keys = views_pos_1.keys()
for key in keys:
    print(key)
    print(views_pos_1[key]["pos"])
    print(views_pos_1[key]["orn_wxyz"])
    print("\n")

view0
[0.29144658970422804, 0.1540829842881886, 0.5513944627325484]
[0.2668073391333377, 0.8887171653008089, 0.36632576241060577, 0.06928982377356051]


view1
[0.5578876977207374, -0.10744477875540048, 0.5430123013772078]
[-0.03894468660134516, 0.9041745877911104, -0.3930714871644215, -0.16262359040633892]


view2
[0.45291522997126205, -0.00619646604332257, 0.5340961850876286]
[0.022248046836998915, 0.9366903069650467, 0.3493662050114398, -0.007716738013556442]


view3
[0.2679255527397324, -0.24442844581216275, 0.37010391852019847]
[-0.07517841530417207, 0.9127177171477998, 0.338701872051414, 0.21581384697402936]




# grid_search

In [2]:
rospy.init_node("pythonnode_register_views",anonymous=True)
rbt = Robot("panda", use_gripper=True)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")

# views_pos = []
# views_orn_wxyz = []
views = {}
view_idx = 0
while True:
    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    cv2.imshow("rgb", rgb)
    key = cv2.waitKey(1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    elif key == ord('s'):
        view_pos = rbt.model.ee_pos_rel()
        view_orn_wxyz = rbt.model.ee_orn_rel()
        views["view"+str(view_idx)] = {"pos": view_pos.tolist(), "orn_wxyz": view_orn_wxyz.tolist()}
        print("view{} saved".format(view_idx))
        view_idx += 1


print(views)

#write pos in json file
with open('config/gridSearch_pos.json', 'w') as outfile:
    json.dump(views, outfile, indent=1)

[WARN] [1688473353.588835]: No controllers are running


rospy already initialized
Camera topic found
view0 saved
view1 saved
view2 saved
view3 saved
{'view0': {'pos': [0.33802430166110387, -0.14284255463766551, 0.6562016104698164], 'orn_wxyz': [0.01843732732826836, 0.8998133959588539, -0.43588034288107935, -0.002060122291974944]}, 'view1': {'pos': [0.05582685336298342, -0.39832176731095076, 0.5386397588024462], 'orn_wxyz': [-0.2928331246988487, 0.8708975206674132, -0.3865031761835437, 0.08000977674318063]}, 'view2': {'pos': [0.612233303535877, -0.05355584428416438, 0.4886439727032261], 'orn_wxyz': [0.13302494214687616, 0.8868473050367259, -0.4413453445656959, -0.03194540884233206]}, 'view3': {'pos': [0.3800200942163088, 0.21520247756653185, 0.5254481651489812], 'orn_wxyz': [0.18273465123874968, 0.8003660038436757, -0.5298807176597906, 0.21271749382761854]}}


In [6]:
#load pos and orn from json file
with open('config/gridSearch_pos.json') as json_file:
    grid_view = json.load(json_file)

keys = grid_view.keys()
for key in keys:
    print(key)
    print(grid_view[key]["pos"])
    print(grid_view[key]["orn_wxyz"])
    print("\n")

view0
[0.29593884231613066, 0.31110448486511083, 0.6175449964239031]
[-0.06909075411989689, 0.48116027816383533, 0.8732609656985465, 0.033563971750770506]


view1
[0.29593884231613066, 0.31110448486511083, 0.6175449964239031]
[-0.06909075411989689, 0.48116027816383533, 0.8732609656985465, 0.033563971750770506]




# Write camera extrinsic

In [10]:
#D415
D415_hand2camera_pos = np.array([0.0488546636437146,-0.03384417860749521,0.0512776975002817]) 
D415_hand2camera_quat = [0.012961267509189803,-0.0012768531849757236,0.7052247395136084,0.708864191484139] #xyzw 
D415_intrinsic = np.array([[925,    0.,           626],
                           [0.,         926,      345],
                           [0.,         0.,           1.]])
#L515
L515_hand2camera_pos = np.array([0.08329189218278059, 0.0014213145240625528, 0.0504764049956106]) 
L515_hand2camera_quat = [0.01521805627198811, 0.00623363612254646, 0.712108725756912, 0.7018765669580811] #xyzw 
L515_intrinsic = np.array([[915.73510742, 0.,           650.41772461],
                           [0.,         916.14764404, 356.6946106],
                           [0.,         0.,           1.]])

camera_calibration = {}
camera_calibration["D415"] = {"pos": D415_hand2camera_pos.tolist(), "quat_xyzw": D415_hand2camera_quat, "intrinsic": D415_intrinsic.tolist()}
camera_calibration["L515"] = {"pos": L515_hand2camera_pos.tolist(), "quat_xyzw": L515_hand2camera_quat, "intrinsic": L515_intrinsic.tolist()}
camera_calibration
#write in a json file
with open('config/camera_calibration.json', 'w') as outfile:
    json.dump(camera_calibration, outfile, indent=1)

In [8]:
#read from camera calin from json file
with open('config/camera_calibration.json') as json_file:
    camera_calibration_read = json.load(json_file)

print(camera_calibration_read["D415"]["pos"])
print(camera_calibration_read["D415"]["quat_xyzw"])
print(camera_calibration_read["D415"]["intrinsic"])

[0.0488546636437146, -0.03384417860749521, 0.0512776975002817]
[0.012961267509189803, -0.0012768531849757236, 0.7052247395136084, 0.708864191484139]
[[925.0, 0.0, 626.0], [0.0, 926.0, 345.0], [0.0, 0.0, 1.0]]


In [9]:
#write pos in json file
with open('config/camera_calibration.json', 'w') as outfile:
    json.dump(views, outfile, indent=1)

In [ ]:
# view_init_pos = [0.17908377, 0.23450877, 0.44783125]
# view_init_orn_wxyz = [0.0030477, 0.92571715, 0.37800879, 0.01215875]
# view_front_pos = [0.36072976, 0.41291873, 0.30648647]
# view_front_orn_wxyz = [ 0.1655683,   0.81494402,  0.28065989, -0.47925298]
# view_right_pos = [0.10118165, 0.47394513, 0.30265239]
# view_right_orn_wxyz = [ 0.3190109,   0.94507174, -0.07051679,  0.00994117]
# view_back_pos = [0.00465632, 0.32838561, 0.26232907]
# view_back_orn_wxyz = [ 0.06700862,  0.96685827, -0.02494526,  0.24509727]

# views_pos = [view_front_pos, view_right_pos, view_back_pos]
# views_orn_wxyz = [view_front_orn_wxyz, view_right_orn_wxyz, view_back_orn_wxyz]

# views = {}
# for i in range(len(views_pos)):
#     views["view"+str(i)] = {"pos": views_pos[i], "orn_wxyz": views_orn_wxyz[i]}
# print(views)